# Mixture Estimation with Truncation

### Density estimation via the GMMis algorithm

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from mvgauss import MvGauss
from mixture import Mixture
from plot import (
    plot_2d_contours, get_2d_confidence_ellipse,
    plot_principal_axes
)

In [ ]:
%matplotlib inline
plt.rcParams['figure.figsize'] = 10, 8

Set up the mixture model we'll use to generate some data:

In [ ]:
c0_pars = pd.DataFrame(index=('mu', 'sigma'))
c0_pars['a'] = 0, 1
c0_pars['b'] = -1, 1.5
c0_corr = {('a', 'b'): -0.8}
c0 = MvGauss.from_correlations(c0_pars, c0_corr)
# c0

In [ ]:
c1_pars = pd.DataFrame(index=('mu', 'sigma'))
c1_pars['a'] = 1.5, 1
c1_pars['b'] = 3.0, 1.5
c1_corr = {('a', 'b'): 0.5}
c1 = MvGauss.from_correlations(c1_pars, c1_corr)

In [ ]:
mix = Mixture(models=[c0, c1],
              weights=[2, 2])
# mix = Mixture(models=[c0, ],
#               weights=[2, ])
mix

In [ ]:
trunc_cutoff = -1.0

In [ ]:
from numpy.random import RandomState
seed = 43
np.random.seed(seed=42)


In [ ]:
def selection_function(sample):
    """Selects deterministically according to second coordinate b:
    Omega = 1    for b > -2
          = 0    for b <= -2
    """
    sample_index =  sample[:,1] > trunc_cutoff
    return sample_index

In [ ]:
def sample_with_truncation(mixture, selection_function, n_samples, iter_max=10):
    truncated_sample = np.zeros((0,mixture.ndim),dtype=float)
    fill_iter = 0
    while (len(truncated_sample) < n_samples) and fill_iter < iter_max:
        untruncated_sample = mixture.joint_sample(int(n_samples*1.5),)
        selection_index = selection_function(untruncated_sample)
        truncated_sample = np.concatenate( (truncated_sample, untruncated_sample[selection_index]))
        fill_iter+=1
    return truncated_sample[:n_samples]
    

In [ ]:
n_samples=400

In [ ]:
sample = sample_with_truncation(mixture=mix, selection_function=selection_function,
                      n_samples=n_samples)

len(sample)

In [ ]:
ax = plt.gca()
ax.scatter(sample[:, 0], sample[:, 1],)
ax.set_title('Truncated sample from GMM')
# ax.hexbin(sample[:,0],sample[:,1],)
for component in mix.models:
    ell = get_2d_confidence_ellipse(component)
    ell.set(fill=False, alpha=1, color='k', ls='-')
    ax.add_artist(ell)
# ax.set_ylim('')
ax.set_aspect(1, 'datalim')
ax.axhline(trunc_cutoff, ls='--', c='r')
# ax.set_aspect(1, adjustable='datalim')


In [ ]:
from sklearn.neighbors.kde import KernelDensity

In [ ]:
kde = KernelDensity(kernel='gaussian', bandwidth=0.75).fit(sample)

In [ ]:
def kde_pdf(sample_coords):
    return np.exp(kde.score_samples(sample_coords))

In [ ]:
from sklearn import mixture

gmm_fixed_n = mixture.GaussianMixture(n_components=len(mix.models))
gmm_fixed_n.fit(sample)

gmm_fixed_n.means_

In [ ]:
gmm_fixed_n.covariances_

In [ ]:
gmm_fixed_n.weights_

In [ ]:
def gmm_fixed_n_pdf(sample_coords):
    return np.exp(gmm_fixed_n.score_samples(sample_coords))

In [ ]:
import pygmmis

In [ ]:
trunc_gmm = pygmmis.GMM(K=2, D=2)
shared_sample = pygmmis.createShared(sample)
trunc_gmm.mean

In [ ]:
w = 0.2    # minimum covariance regularization, same units as data
gmmis_cutoff = 4 # segment the data set into neighborhood within 5 sigma around components
tol = 1e-4 # tolerance on logL to terminate EM
pygmmis.VERBOSITY = 2      # 0,1,2
pygmmis.OVERSAMPLING = 10  # number of imputation samples per data sample

gmmis_seed= 1066
rng = RandomState(gmmis_seed)

In [ ]:
def initFromGmmAndRelax(gmm, data, covar=None, s=None, k=None, rng=np.random):
    pygmmis.initFromSimpleGMM(gmm, data, covar, s, k, rng)
    gmm.covar += 1.5*trunc_gmm.covar*np.eye(2)

In [ ]:
logL, U = pygmmis.fit(trunc_gmm, shared_sample,
                      init_callback=pygmmis.initFromSimpleGMM,
#                       init_callback=initFromGmmAndRelax,
                      sel_callback=selection_function, covar_callback=None, w=w, cutoff=gmmis_cutoff,
                      background=None, tol=tol, rng=rng)

In [ ]:
gmmis_components = []
for idx in range(len(trunc_gmm.mean)):
    mean = pd.Series(trunc_gmm.mean[idx], name='mu')
    sigma = pd.Series(np.diagonal(trunc_gmm.covar[idx]),name='sigma')**0.5
    pars = pd.concat([mean,sigma], axis=1).T
    gmmis_components.append(MvGauss(pars =pars, cov=trunc_gmm.covar[idx]))
    
    
gmmis_mix = Mixture(gmmis_components, trunc_gmm.amp)

In [ ]:
# gmmis_mix

In [ ]:
# mix

In [ ]:
xlim=ylim=(-5,6)

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(20, 5))
# for ax in axes.flat:
#     im = ax.imshow(np.random.random((10,10)), vmin=0, vmax=1)

levels = np.linspace(0., .085, num=10)
ax_idx = 0

# plot_2d_contours(kde_pdf, xlim, ylim, ax=axes[ax_idx],
#                  #                  levels=levels
#                  )
# axes[ax_idx].set_title('KDE')
# ax_idx += 1

plot_2d_contours(gmm_fixed_n_pdf, xlim, ylim, ax=axes[ax_idx])
axes[ax_idx].set_title('GMM (fixed N)')
ax_idx += 1


# plot_2d_contours(gmm_bayes_pdf, xlim, ylim, ax=axes[ax_idx])
# axes[ax_idx].set_title('GMM (Bayes)')
# ax_idx += 1

cset, _ = plot_2d_contours(mix.joint_pdf, xlim, ylim, ax=axes[ax_idx],)
# plt.colorbar(cset,ax=axes.tolist())
axes[ax_idx].set_title('True PDF')
ax_idx += 1

cset, _ = plot_2d_contours(gmmis_mix.joint_pdf, xlim, ylim, ax=axes[ax_idx],)
# plt.colorbar(cset,ax=axes.tolist())
axes[ax_idx].set_title('GMMis PDF')

# fig.subplots_adjust(right=0.8)
# cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
# fig.colorbar(im, cax=cbar_ax)
for ax in axes:
    ax.set_aspect('equal', 'box-forced')
    ax.axhline(y=trunc_cutoff, ls='--', c='r')

print("N samples:", n_samples)